In [9]:
%matplotlib inline
import dgl
import glob
import pprint
import numpy as np
import awkward as ak
import networkx as nx
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from os import path
from tqdm import tqdm
from pathlib import Path
from trainresults import TrainResults
from train_eval_func import train, evaluate
from copy import deepcopy
from dgl.data import DGLDataset
from dgl.dataloading import GraphDataLoader
from TauGraphDatasetInfo import TauGraphDatasetInfo
from TauGraphDataset import TauGraphDataset, GetNodeFeatureVectors, GetEdgeFeatureVectors
from TauGraphDataset import GetNeighborNodes, GetEdgeList, GetEdgeFeatureVectorsFromSourceNode, Graph2FlatZeropaddedList

plt.rcParams.update({'font.size': 20})
plt.rcParams['text.usetex'] = True
lw = 2
xyLabelFontSize = 20
xLabelPad = 10
yLabelPad = 15
pp = pprint.PrettyPrinter()

In [2]:
def getDatasetNames(datasetDir):
    files = glob.glob(datasetDir + '/*.json', recursive=True)
    files.sort()
    datasetDirectories = [path.dirname(file) for file in files]
    datasetnames = [path.normpath(dir).split(path.sep)[-1] for dir in datasetDirectories]
    return datasetDirectories, datasetnames

In [3]:
datasetDir = '/ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets'
datasetDirs, datasetNames = getDatasetNames(datasetDir)
print(datasetDirs)
print(datasetNames)

['/ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets']
['Graphs_DYJetsToLL_M-50_genuineTaus_and_jets']


In [4]:
dataset = TauGraphDataset(datasetNames[0], datasetDirs[0])
print(dataset)

Done loading data from cached files.


In [5]:
print(f'name: {datasetNames[0]},\n directory: {datasetDirs[0]}')
graph, label = dataset[0]
print(graph)
print(f'label: {label}')
print(f'graph classes: {dataset.graphClasses}')
print(f'dataset graph count: {dataset.num_graphs}')
print(f'nodeFeatKeys: {dataset.nodeFeatKeys}')
print(f'edgeFeatKeys: {dataset.edgeFeatKeys}')
print(f'graphFeatkeys: {dataset.graphFeatKeys}')
print(f'max node count: {dataset.maxNodeCount}')
print(f'min node count: {dataset.minNodeCount}')

name: Graphs_DYJetsToLL_M-50_genuineTaus_and_jets,
 directory: /ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets
Graph(num_nodes=25, num_edges=600,
      ndata_schemes={'feat': Scheme(shape=(7,), dtype=torch.float64)}
      edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.float32)})
label: 1
graph classes: ['0', '1']
dataset graph count: 200000
nodeFeatKeys: ['pt', 'eta', 'phi', 'mass', 'charge', 'particleType', 'summand']
edgeFeatKeys: ['deltaEta', 'deltaPhi', 'deltaR']
graphFeatkeys: ['nodeCount']
max node count: 81
min node count: 2


In [6]:
graphs, labels = dataset[:]
g = graphs[0]
nFeatDim = dataset.dim_nfeats
eFeatDim = dataset.dim_efeats
maxNodeCount = dataset.maxNodeCount
print(g)
print(f'nfeatDim: {nFeatDim}')
print(f'eFeatDim: {eFeatDim}')
print(f'maxNodeCount: {maxNodeCount}')
nodeAndEdgeFeaturePaddedDim = nFeatDim + eFeatDim * (maxNodeCount - 1)
print(f'nodeAndEdgeFeaturePaddedDim: {nodeAndEdgeFeaturePaddedDim}')
inputSize = nodeAndEdgeFeaturePaddedDim * maxNodeCount
print(f'whole inputsize: {inputSize}')

useEdgeFeat = True

temp = Graph2FlatZeropaddedList(g, nFeatDim, eFeatDim, maxNodeCount, useEdgeFeat)
print(len(temp))
print(temp)

Graph(num_nodes=25, num_edges=600,
      ndata_schemes={'feat': Scheme(shape=(7,), dtype=torch.float64)}
      edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.float32)})
nfeatDim: 7
eFeatDim: 3
maxNodeCount: 81
nodeAndEdgeFeaturePaddedDim: 247
whole inputsize: 20007
20007
[tensor(0.6792, dtype=torch.float64), tensor(1.2382, dtype=torch.float64), tensor(-0.6319, dtype=torch.float64), tensor(0.1395, dtype=torch.float64), tensor(1., dtype=torch.float64), tensor(1., dtype=torch.float64), tensor(1., dtype=torch.float64), tensor(0.2256), tensor(0.0218), tensor(0.2266), tensor(0.2287), tensor(-0.2580), tensor(0.3448), tensor(0.2360), tensor(-0.3103), tensor(0.3898), tensor(0.2472), tensor(-0.1631), tensor(0.2962), tensor(0.2487), tensor(-0.2702), tensor(0.3672), tensor(0.2551), tensor(-0.2399), tensor(0.3502), tensor(0.2670), tensor(0.2110), tensor(0.3403), tensor(0.3505), tensor(-0.0533), tensor(0.3545), tensor(0.6502), tensor(-0.2721), tensor(0.7049), tensor(0.5583), tensor(-0.5040), 

In [21]:
def getInputData(dgldataset, useEdgeFeatures):
    inputs = [] 
    graphs, labels = dgldataset[:]
    maxNodeCount = dgldataset.maxNodeCount
    nFeatDim = dgldataset.dim_nfeats
    eFeatDim = dgldataset.dim_efeats
    
    import time
    start = time.time()
    it = 0
    
    for i in tqdm(range(len(graphs))):
        inputs.append(Graph2FlatZeropaddedList(graphs[i], nFeatDim, eFeatDim, maxNodeCount, useEdgeFeatures))

    # Stack all inputs_ vertically
    print(type(inputs))
    inputs = np.array(inputs, dtype=np.float32)
    print(inputs)
    print(type(inputs))
    inputs = np.vstack(inputs)
    print(inputs)
    

    # Stack all labels_ horizontally
    labels = np.hstack(labels)

    print("Input shape: ", inputs.shape)
    print("Labels shape: ", labels.shape)

    labels = tf.keras.utils.to_categorical(labels)
    print(labels.shape)
    print(labels[0])
    end = time.time() - start
    print(f'graphs to flattened zero padded list took {end:.2f} seconds ({end/60:.2f} minutes)')
    return inputs, labels

In [23]:
inputs, labels = getInputData(dataset, False)

 21%|██        | 41702/200000 [00:10<00:38, 4085.96it/s]


KeyboardInterrupt: 

datasetName = datasetNames[0]
datasetDir = datasetDirs[0]
dataset = TauGraphDataset(datasetName, datasetDir)
dataset.printProperties()

graph, label = dataset[0]
print(graph)
print(f'Label: {label}')
print(GetNodeFeatureVectors(graph))

In [ ]:
import time
now = time.time()

batchSize = 64
print(f'Device: {device}')

for i in range(len(datasetDirs)):
    dataset = TauGraphDataset(datasetNames[i], datasetDirs[i])
    splitIndices = dataset.get_split_indices()

    train_sampler = SubsetRandomSampler(splitIndices['train'])
    test_sampler = SubsetRandomSampler(splitIndices['test'])

    train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=batchSize, drop_last=False)
    test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=batchSize, drop_last=False)
    
    # Create the model with given dimensions
    model = MPGNN(
        dataset.dim_nfeats, dataset.dim_efeats, 
        node_out_feats=16, edge_hidden_feats=16, 
        num_step_message_passing=2, n_classes=dataset.num_graph_classes).to(device)
    
    model.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()
    epochs = 10
    
    # train
    print(f'Beginning training on dataset {datasetNames[i]}')
    results, bestmodel = trainEpochs(model, device, train_dataloader, optimizer, loss_fn, batchSize, epochs)
    results.printBestResult()
    
    # save results
    outputFolder = path.join(datasetDirs[i], 'Output_MPGNN_LR0001')
    Path(outputFolder).mkdir(parents=True, exist_ok=True)
    
    results.savePlots(outputFolder)
    results.dumpSummary(outputFolder)
    results.pickledump(outputFolder)
    
    # save the best model for inference. (when loading for inference -> model.eval()!! )
    # https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict
    torch.save(bestmodel.state_dict(), path.join(outputFolder, 'model.pt'))
    
    print(f'------------------({i+1}/{len(datasetDirs)}) models trained------------------\n')


end = time.time()
elapsed = end - now
print(f'{elapsed} seconds elapsed')